## Open Weather Map API

Purpose is to retrieve data from Open Weather Map API (https://openweathermap.org/api/one-call-3#start) for building Ciampino Digital Model and getting weather data for it.

### Setting API Key

Here is reported API Key for performing calls.

In [50]:
API_KEY = '997b847a9ed5dda58598e1f0015be1e6'
CITY = 'Ciampino,Lazio,IT'

Then, it's used Geocoding API from Open Weather Map to retrieve latitude and longitude of inspected city, which is Ciampino.
But first, import all necessary libraries.

In [19]:
import requests
import datetime

In [31]:
# Geocoding API Url

def geocode_city():
    return f'http://api.openweathermap.org/geo/1.0/direct?q={CITY}&limit=5&appid={API_KEY}'

In [32]:
# Data retrieved from Geocoding API url and then latitude and longitude stored
def get_city_data():
    res = requests.get(geocode_city()).json()
    return {'lat': res[0]['lat'], 'lon': res[0]['lon']}

city_lat = get_city_data()['lat']
city_lon = get_city_data()['lon']

Now it's possible to make first call with Open Weather Map API for weather.

In [ ]:
def get_weather_data(url):
    return requests.get(url).json()

In [46]:
# API for Current Weather Data ONLY!
url_curr_weather_api = f'https://api.openweathermap.org/data/2.5/weather?lat={city_lat}&lon={city_lon}&&appid={API_KEY}&units=metric'
print(get_weather_data(url_curr_weather_api))

{'coord': {'lon': 12.6005, 'lat': 41.8003}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 10.16, 'feels_like': 8.88, 'temp_min': 7.25, 'temp_max': 11.22, 'pressure': 1027, 'humidity': 63, 'sea_level': 1027, 'grnd_level': 1009}, 'visibility': 10000, 'wind': {'speed': 3.09, 'deg': 70}, 'clouds': {'all': 0}, 'dt': 1735325591, 'sys': {'type': 1, 'id': 6792, 'country': 'IT', 'sunrise': 1735281365, 'sunset': 1735314316}, 'timezone': 3600, 'id': 3178738, 'name': 'Ciampino', 'cod': 200}


In [47]:
# Re-elaborate data from API response for setting correct date (not Unix timestamps)
def set_date(url):
    data = get_weather_data(url)
    data['dt'] = datetime.fromtimestamp(data['dt']).strftime('%Y-%m-%d %H:%M:%S')
    data['sys']['sunrise'] = datetime.fromtimestamp(data['sys']['sunrise']).strftime('%Y-%m-%d %H:%M:%S')
    data['sys']['sunset'] = datetime.fromtimestamp(data['sys']['sunset']).strftime('%Y-%m-%d %H:%M:%S')
    return data

In [48]:
print(set_date(url_curr_weather_api))

{'coord': {'lon': 12.6005, 'lat': 41.8003}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 10.16, 'feels_like': 8.88, 'temp_min': 7.25, 'temp_max': 11.22, 'pressure': 1027, 'humidity': 63, 'sea_level': 1027, 'grnd_level': 1009}, 'visibility': 10000, 'wind': {'speed': 3.09, 'deg': 70}, 'clouds': {'all': 0}, 'dt': '2024-12-27 19:53:11', 'sys': {'type': 1, 'id': 6792, 'country': 'IT', 'sunrise': '2024-12-27 07:36:05', 'sunset': '2024-12-27 16:45:16'}, 'timezone': 3600, 'id': 3178738, 'name': 'Ciampino', 'cod': 200}


In [52]:
url_data_wea = f'https://api.openweathermap.org/data/3.0/onecall?lat={city_lat}&lon={city_lon}&appid={API_KEY}'
rrr = get_weather_data(url_data_wea)
rrr
# to continue -> see open weather map api email

{'cod': 401,
 'message': 'Please note that using One Call 3.0 requires a separate subscription to the One Call by Call plan. Learn more here https://openweathermap.org/price. If you have a valid subscription to the One Call by Call plan, but still receive this error, then please see https://openweathermap.org/faq#error401 for more info.'}